paddle版波士顿房价

In [1]:
#加载飞桨、Numpy和相关类库
import paddle
from paddle.nn import Linear
import paddle.nn.functional as F
import numpy as np
import os
import random

F:\Anaconda\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


## 数据处理

In [2]:
def load_data():
    # 从文件导入数据
    datafile = './data/housing.data'
    data = np.fromfile(datafile, sep=' ', dtype=np.float32)

    # 每条数据包括14项，其中前面13项是影响因素，第14项是相应的房屋价格中位数
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', \
                      'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_num = len(feature_names)

    # 将原始数据进行Reshape，变成[N, 14]这样的形状
    data = data.reshape([data.shape[0] // feature_num, feature_num])

    # 将原数据集拆分成训练集和测试集
    # 这里使用80%的数据做训练，20%的数据做测试
    # 测试集和训练集必须是没有交集的
    ratio = 0.8
    offset = int(data.shape[0] * ratio)
    training_data = data[:offset]

    # 计算train数据集的最大值，最小值，平均值
    maximums, minimums, avgs, std = training_data.max(axis=0), training_data.min(axis=0), training_data.sum(axis=0) / training_data.shape[0], training_data.std(axis = 0)
    # 记录数据的归一化参数，在预测时对数据做归一化
    global max_values
    global min_values
    global avg_values
    global std_values
    max_values = maximums
    min_values = minimums
    avg_values = avgs
    std_values = std

    # 对数据进行归一化处理
    for i in range(feature_num):
        data[:, i] = (data[:, i] - avgs[i]) / std[i]

    # 训练集和测试集的划分比例
    training_data = data[:offset]
    test_data = data[offset:]
    return training_data, test_data


In [2]:
class Regressor(paddle.nn.Layer):
    # self代表类的实例自身
    def __init__(self):
        # 初始化父类中的一些参数
        super(Regressor, self).__init__()
        
        # 定义一层全连接层，输入维度是13，输出维度是1
        self.fc1 = Linear(in_features=13, out_features=5)
        self.fc2 = Linear(in_features=5, out_features=1)
    

    def Relu(self,x):
        return np.where(x < 0,0,x)


    # 网络的前向计算
    def forward(self, inputs):
        x = self.fc1(inputs)
        x = self.Relu(x)
        x = self.fc2(x)
        return x

In [4]:
def train():
    # 声明定义好的线性回归模型
    model = Regressor()
    # 开启模型训练模式
    model.train()
    # 加载数据
    training_data, test_data = load_data()
    # 定义优化算法，使用随机梯度下降SGD
    # 学习率设置为0.01
    opt = paddle.optimizer.SGD(learning_rate=0.01, parameters=model.parameters())

    EPOCH_NUM = 10   # 设置外层循环次数
    BATCH_SIZE = 10  # 设置batch大小

    # 定义外层循环
    for epoch_id in range(EPOCH_NUM):
        # 在每轮迭代开始之前，将训练数据的顺序随机的打乱
        np.random.shuffle(training_data)
        # 将训练数据进行拆分，每个batch包含10条数据
        mini_batches = [training_data[k:k+BATCH_SIZE] for k in range(0, len(training_data), BATCH_SIZE)]
        # 定义内层循环
        for iter_id, mini_batch in enumerate(mini_batches):
            x = np.array(mini_batch[:, :-1]) # 获得当前批次训练数据
            y = np.array(mini_batch[:, -1:]) # 获得当前批次训练标签（真实房价）
            # 将numpy数据转为飞桨动态图tensor形式
            house_features = paddle.to_tensor(x)
            prices = paddle.to_tensor(y)
            
            # 前向计算
            predicts = model(house_features)
            
            # 计算损失
            loss = F.square_error_cost(predicts, label=prices)
            avg_loss = paddle.mean(loss)
            if iter_id%20==0:
                print("epoch: {}, iter: {}, loss is: {}".format(epoch_id, iter_id, avg_loss.numpy()))
            
            # 反向传播
            avg_loss.backward()
            # 最小化loss,更新参数
            opt.step()
            # 清除梯度
            opt.clear_grad()

    # 保存模型参数，文件名为LR_model.pdparams
    paddle.save(model.state_dict(), 'LR_model.pdparams')
    print("模型保存成功，模型参数保存在LR_model.pdparams中")

In [5]:
def load_one_example():
    training_data, test_data = load_data()
    # 从上边已加载的测试集中，随机选择一条作为测试数据
    idx = np.random.randint(0, test_data.shape[0])
    idx = -10
    one_data, label = test_data[idx, :-1], test_data[idx, -1]
    # 修改该条数据shape为[1,13]
    one_data =  one_data.reshape([1,-1])

    return one_data, label

epoch: 0, iter: 0, loss is: [0.08927596]
epoch: 0, iter: 20, loss is: [0.04351359]
epoch: 0, iter: 40, loss is: [0.06200176]
epoch: 1, iter: 0, loss is: [0.02089665]
epoch: 1, iter: 20, loss is: [0.08222677]
epoch: 1, iter: 40, loss is: [0.20478992]
epoch: 2, iter: 0, loss is: [0.04145242]
epoch: 2, iter: 20, loss is: [0.15099502]
epoch: 2, iter: 40, loss is: [0.02584986]
epoch: 3, iter: 0, loss is: [0.05165129]
epoch: 3, iter: 20, loss is: [0.12128074]
epoch: 3, iter: 40, loss is: [0.01223269]
epoch: 4, iter: 0, loss is: [0.0389842]
epoch: 4, iter: 20, loss is: [0.07199501]
epoch: 4, iter: 40, loss is: [0.03861689]
epoch: 5, iter: 0, loss is: [0.0647126]
epoch: 5, iter: 20, loss is: [0.03957129]
epoch: 5, iter: 40, loss is: [0.09043328]
epoch: 6, iter: 0, loss is: [0.08471931]
epoch: 6, iter: 20, loss is: [0.06079114]
epoch: 6, iter: 40, loss is: [0.01805439]
epoch: 7, iter: 0, loss is: [0.01560107]
epoch: 7, iter: 20, loss is: [0.02061713]
epoch: 7, iter: 40, loss is: [0.04313775]
ep

In [7]:
if __name__ == '__main__':
    train()
    validation()